docker run -d \
  --name qdrant \
  -p 6333:6333 \
  qdrant/qdrant

In [1]:
from langchain_community.vectorstores import Qdrant
from langchain_qdrant import QdrantVectorStore
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from tqdm import tqdm
import os

HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")

def load_and_chunk_data(data_path):
    docs = []
    # Load all .txt files from the specified folder and its subfolders
    for root, _, files in os.walk(data_path):
        for filename in files:
            if filename.endswith('.txt'):
                file_path = os.path.join(root, filename)
                loader = TextLoader(file_path, encoding='utf-8')
                docs.extend(loader.load())

    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=True
    )

    chunk_size = 512
    chunk_overlap = 0

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )
    chunked_docs = []

    for doc in docs:
        md_header_splits = markdown_splitter.split_text(doc.page_content)
        chunked_docs.extend(text_splitter.split_documents(md_header_splits))

    return chunked_docs




/tmp/ipykernel_32520/1995128018.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  HF_EMBEDDING = HuggingFaceEmbeddings(model_name="keepitreal/vietnamese-sbert")
/home/justtuananh/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/justtuananh/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible

In [2]:
data_path = '/home/justtuananh/AI4TUAN/DOAN2024/eval_rag_vietnamese/data/thongtintuyensinh'
docs = load_and_chunk_data(data_path)

In [3]:
print(len(docs))
for doc in docs: 
    print(doc)


59
page_content='Thực hiện theo quy định của Bộ Quốc phòng. Học viên được học tập, khám phá và làm chủ các công nghệ tiên tiến được ứng dụng trong quân sự. Học viên được bảo đảm mọi mặt về sinh hoạt và đào tạo trong suốt thời gian đào tạo, sau khi tốt nghiệp được Bộ Quốc phòng phân công công tác tại các đơn vị trong toàn quân, thân nhân học viên (bố, mẹ, con) được hưởng chế độ bảo hiểm y tế theo quy định.' metadata={'Header 1': '2. Đối tượng đào tạo, ngành đào tạo', 'Header 2': '2.1. Đối tượng đào tạo trình độ đại học'}
page_content='- Trình độ đào tạo đại học được đào tạo theo 50 chuyên ngành thuộc các lĩnh vực: Công nghệ thông tin, trí tuệ nhân tạo, khoa học máy tính, công nghệ rô bốt, UAV, điện tử - viễn thông, kỹ thuật điều khiển tự động và tự động hoá, kỹ thuật ra đa dẫn đường, kỹ thuật cơ khí động lực, kỹ thuật xây dựng, công nghệ hoá học, môi trường, kỹ thuật xây dựng công trình giao thông, công nghệ vật liệu, Kỹ thuật trắc địa - bản đồ, …và 02 chuyên ngành đào tạo kỹ sư chất lư

In [6]:
from dotenv import load_dotenv
load_dotenv()


url = os.getenv("URL_QDRANT")
api_key = os.getenv("API_QDRANT")

qdrant = QdrantVectorStore.from_documents(
    docs,
    HF_EMBEDDING,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="thongtintuyensinh_db",
)

In [8]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="84627acc-a45c-44cd-b9c5-e45053dff2f3.europe-west3-0.gcp.cloud.qdrant.io:6333", 
    api_key=api_key,
)

print(qdrant_client.get_collections())

UnexpectedResponse: Unexpected Response: 403 (Forbidden)
Raw response content:
b'{"error":"forbidden"}'